In [1]:
import numpy as np
import cv2
import os
import torch as t
import visdom
import time

from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils import data

vis = visdom.Visdom(env = 'grad_FC_txt')


In [2]:
kernel_ = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

p = np.zeros((28,3,3))
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,1] = 1; p[0] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,2] = 1; p[1] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,0] = 1; p[2] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,2] = 1; p[3] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,0] = 1; p[4] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,1] = 1; p[5] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,2] = 1; p[6] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[0,2] = 1; p[7] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,0] = 1; p[8] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,2] = 1; p[9] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,0] = 1; p[10] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,1] = 1; p[11] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,2] = 1; p[12] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,0] = 1; p[13] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,2] = 1; p[14] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,0] = 1; p[15] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,1] = 1; p[16] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,2] = 1; p[17] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[1,2] = 1; p[18] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,0] = 1; p[19] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,1] = 1; p[20] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,2] = 1; p[21] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,0] = 1; p[22] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,1] = 1; p[23] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,2] = 1; p[24] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,1] = 1; p[25] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,2] = 1; p[26] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,1] = 1;zeros[2,2] = 1; p[27] = zeros


In [ ]:
img_2grad = np.zeros((28,30,30,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_1"
save_path = "C:\\newfolder43_1_txt7"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                #image1 = image1/255.0
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                #image_gaussion = np.true_divide(image_gaussion,255.0)
                for m in range(28):
                    img_2grad[m] = (cv2.filter2D(image_gaussion,cv2.CV_16S,p[m]))/255.0
                    
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    #aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    #aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    #aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    #var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                #all_aver_array = np.hstack((all_aver_array,aver_array))   
                #all_var_array1 = np.hstack((all_var_array1,var_array1))
                #all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        
        
        #result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in all_var_array))
        fw.close()


D:\newfolder43_1\clear_face\picture100_147_.png
D:\newfolder43_1\clear_face\picture100_1595_.png
D:\newfolder43_1\clear_face\picture100_1599_.png
D:\newfolder43_1\clear_face\picture100_1601_.png
D:\newfolder43_1\clear_face\picture100_1614_.png
D:\newfolder43_1\clear_face\picture100_1623_.png
D:\newfolder43_1\clear_face\picture100_1626_.png
D:\newfolder43_1\clear_face\picture100_1648_.png
D:\newfolder43_1\clear_face\picture100_1710_.png
D:\newfolder43_1\clear_face\picture100_1712_.png
D:\newfolder43_1\clear_face\picture100_1714_.png
D:\newfolder43_1\clear_face\picture100_1720_.png
D:\newfolder43_1\clear_face\picture100_1723_.png
D:\newfolder43_1\clear_face\picture100_1726_.png
D:\newfolder43_1\clear_face\picture100_1741_.png
D:\newfolder43_1\clear_face\picture100_1745_.png
D:\newfolder43_1\clear_face\picture100_1751_.png
D:\newfolder43_1\clear_face\picture100_1809_.png
D:\newfolder43_1\clear_face\picture100_1813_.png
D:\newfolder43_1\clear_face\picture100_1815_.png
D:\newfolder43_1\clea

D:\newfolder43_1\clear_face\picture100_4538_.png
D:\newfolder43_1\clear_face\picture100_453_.png
D:\newfolder43_1\clear_face\picture100_4566_.png
D:\newfolder43_1\clear_face\picture100_4568_.png
D:\newfolder43_1\clear_face\picture100_4590_.png
D:\newfolder43_1\clear_face\picture100_4624_.png
D:\newfolder43_1\clear_face\picture100_4626_.png
D:\newfolder43_1\clear_face\picture100_4635_.png
D:\newfolder43_1\clear_face\picture100_4637_.png
D:\newfolder43_1\clear_face\picture100_463_.png
D:\newfolder43_1\clear_face\picture100_4641_.png
D:\newfolder43_1\clear_face\picture100_4643_.png
D:\newfolder43_1\clear_face\picture100_4645_.png
D:\newfolder43_1\clear_face\picture100_4646_.png
D:\newfolder43_1\clear_face\picture100_4670_.png
D:\newfolder43_1\clear_face\picture100_4672_.png
D:\newfolder43_1\clear_face\picture100_4677_.png
D:\newfolder43_1\clear_face\picture100_4679_.png
D:\newfolder43_1\clear_face\picture100_4681_.png
D:\newfolder43_1\clear_face\picture100_4684_.png
D:\newfolder43_1\clear

D:\newfolder43_1\clear_face\picture100_584_.png
D:\newfolder43_1\clear_face\picture100_5850_.png
D:\newfolder43_1\clear_face\picture100_5851_.png
D:\newfolder43_1\clear_face\picture100_5852_.png
D:\newfolder43_1\clear_face\picture100_585_.png
D:\newfolder43_1\clear_face\picture100_586_.png
D:\newfolder43_1\clear_face\picture100_587_.png
D:\newfolder43_1\clear_face\picture100_6009_.png
D:\newfolder43_1\clear_face\picture100_6011_.png
D:\newfolder43_1\clear_face\picture100_6013_.png
D:\newfolder43_1\clear_face\picture100_6014_.png
D:\newfolder43_1\clear_face\picture100_6015_.png
D:\newfolder43_1\clear_face\picture100_6016_.png
D:\newfolder43_1\clear_face\picture100_6017_.png
D:\newfolder43_1\clear_face\picture100_6018_.png
D:\newfolder43_1\clear_face\picture100_6030_.png
D:\newfolder43_1\clear_face\picture100_6031_.png
D:\newfolder43_1\clear_face\picture100_6032_.png
D:\newfolder43_1\clear_face\picture100_6033_.png
D:\newfolder43_1\clear_face\picture100_6034_.png
D:\newfolder43_1\clear_f

D:\newfolder43_1\clear_face\picture1_11358_.png
D:\newfolder43_1\clear_face\picture1_11360_.png
D:\newfolder43_1\clear_face\picture1_11363_.png
D:\newfolder43_1\clear_face\picture1_11365_.png
D:\newfolder43_1\clear_face\picture1_11368_.png
D:\newfolder43_1\clear_face\picture1_11370_.png
D:\newfolder43_1\clear_face\picture1_11372_.png
D:\newfolder43_1\clear_face\picture1_11374_.png
D:\newfolder43_1\clear_face\picture1_11376_.png
D:\newfolder43_1\clear_face\picture1_11378_.png
D:\newfolder43_1\clear_face\picture1_1137_.png
D:\newfolder43_1\clear_face\picture1_11380_.png
D:\newfolder43_1\clear_face\picture1_11382_.png
D:\newfolder43_1\clear_face\picture1_11391_.png
D:\newfolder43_1\clear_face\picture1_1139_.png
D:\newfolder43_1\clear_face\picture1_11421_.png
D:\newfolder43_1\clear_face\picture1_11422_.png
D:\newfolder43_1\clear_face\picture1_11424_.png
D:\newfolder43_1\clear_face\picture1_11427_.png
D:\newfolder43_1\clear_face\picture1_11428_.png
D:\newfolder43_1\clear_face\picture1_1142_

D:\newfolder43_1\clear_face\picture1_1400_.png
D:\newfolder43_1\clear_face\picture1_1403_.png
D:\newfolder43_1\clear_face\picture1_1407_.png
D:\newfolder43_1\clear_face\picture1_14086_.png
D:\newfolder43_1\clear_face\picture1_1409_.png
D:\newfolder43_1\clear_face\picture1_1420_.png
D:\newfolder43_1\clear_face\picture1_1421_.png
D:\newfolder43_1\clear_face\picture1_1424_.png
D:\newfolder43_1\clear_face\picture1_14280_.png
D:\newfolder43_1\clear_face\picture1_14281_.png
D:\newfolder43_1\clear_face\picture1_14282_.png
D:\newfolder43_1\clear_face\picture1_14283_.png
D:\newfolder43_1\clear_face\picture1_1429_.png
D:\newfolder43_1\clear_face\picture1_1432_.png
D:\newfolder43_1\clear_face\picture1_14713_.png
D:\newfolder43_1\clear_face\picture1_14716_.png
D:\newfolder43_1\clear_face\picture1_14718_.png
D:\newfolder43_1\clear_face\picture1_14721_.png
D:\newfolder43_1\clear_face\picture1_14725_.png
D:\newfolder43_1\clear_face\picture1_14727_.png
D:\newfolder43_1\clear_face\picture1_14729_.png
D

...

D:\newfolder43_1\clear_face\picture90_1174_.png
D:\newfolder43_1\clear_face\picture90_11750_.png
D:\newfolder43_1\clear_face\picture90_11758_.png
D:\newfolder43_1\clear_face\picture90_11759_.png
D:\newfolder43_1\clear_face\picture90_1176_.png
D:\newfolder43_1\clear_face\picture90_11782_.png
D:\newfolder43_1\clear_face\picture90_1178_.png
D:\newfolder43_1\clear_face\picture90_1180_.png
D:\newfolder43_1\clear_face\picture90_11839_.png
D:\newfolder43_1\clear_face\picture90_11844_.png
D:\newfolder43_1\clear_face\picture90_11845_.png
D:\newfolder43_1\clear_face\picture90_11846_.png
D:\newfolder43_1\clear_face\picture90_11886_.png
D:\newfolder43_1\clear_face\picture90_11889_.png
D:\newfolder43_1\clear_face\picture90_11890_.png
D:\newfolder43_1\clear_face\picture90_11893_.png
D:\newfolder43_1\clear_face\picture90_11894_.png
D:\newfolder43_1\clear_face\picture90_11895_.png
D:\newfolder43_1\clear_face\picture90_11896_.png
D:\newfolder43_1\clear_face\picture90_11897_.png
D:\newfolder43_1\clear_f

D:\newfolder43_1\clear_face\picture90_3282_.png
D:\newfolder43_1\clear_face\picture90_3283_.png
D:\newfolder43_1\clear_face\picture90_3285_.png
D:\newfolder43_1\clear_face\picture90_3287_.png
D:\newfolder43_1\clear_face\picture90_3303_.png
D:\newfolder43_1\clear_face\picture90_3304_.png
D:\newfolder43_1\clear_face\picture90_3306_.png
D:\newfolder43_1\clear_face\picture90_3309_.png
D:\newfolder43_1\clear_face\picture90_3314_.png
D:\newfolder43_1\clear_face\picture90_3317_.png
D:\newfolder43_1\clear_face\picture90_3321_.png
D:\newfolder43_1\clear_face\picture90_3324_.png
D:\newfolder43_1\clear_face\picture90_3329_.png
D:\newfolder43_1\clear_face\picture90_3331_.png
D:\newfolder43_1\clear_face\picture90_3333_.png
D:\newfolder43_1\clear_face\picture90_3334_.png
D:\newfolder43_1\clear_face\picture90_3336_.png
D:\newfolder43_1\clear_face\picture90_3339_.png
D:\newfolder43_1\clear_face\picture90_3340_.png
D:\newfolder43_1\clear_face\picture90_3343_.png
D:\newfolder43_1\clear_face\picture90_33

D:\newfolder43_1\clear_face\picture90_4324_.png
D:\newfolder43_1\clear_face\picture90_4326_.png
D:\newfolder43_1\clear_face\picture90_4328_.png
D:\newfolder43_1\clear_face\picture90_4330_.png
D:\newfolder43_1\clear_face\picture90_4376_.png
D:\newfolder43_1\clear_face\picture90_4406_.png
D:\newfolder43_1\clear_face\picture90_442_.png
D:\newfolder43_1\clear_face\picture90_4457_.png
D:\newfolder43_1\clear_face\picture90_4467_.png
D:\newfolder43_1\clear_face\picture90_4469_.png
D:\newfolder43_1\clear_face\picture90_4471_.png
D:\newfolder43_1\clear_face\picture90_4473_.png
D:\newfolder43_1\clear_face\picture90_4475_.png
D:\newfolder43_1\clear_face\picture90_4478_.png
D:\newfolder43_1\clear_face\picture90_451_.png
D:\newfolder43_1\clear_face\picture90_4527_.png
D:\newfolder43_1\clear_face\picture90_4590_.png
D:\newfolder43_1\clear_face\picture90_4595_.png
D:\newfolder43_1\clear_face\picture90_4600_.png
D:\newfolder43_1\clear_face\picture90_4702_.png
D:\newfolder43_1\clear_face\picture90_4704

D:\newfolder43_1\clear_face\picture90_6118_.png
D:\newfolder43_1\clear_face\picture90_6121_.png
D:\newfolder43_1\clear_face\picture90_6133_.png
D:\newfolder43_1\clear_face\picture90_6388_.png
D:\newfolder43_1\clear_face\picture90_6390_.png
D:\newfolder43_1\clear_face\picture90_6391_.png
D:\newfolder43_1\clear_face\picture90_6404_.png
D:\newfolder43_1\clear_face\picture90_6406_.png
D:\newfolder43_1\clear_face\picture90_6432_.png
D:\newfolder43_1\clear_face\picture90_6434_.png
D:\newfolder43_1\clear_face\picture90_6462_.png
D:\newfolder43_1\clear_face\picture90_6477_.png
D:\newfolder43_1\clear_face\picture90_6516_.png
D:\newfolder43_1\clear_face\picture90_6556_.png
D:\newfolder43_1\clear_face\picture90_6563_.png
D:\newfolder43_1\clear_face\picture90_6570_.png
D:\newfolder43_1\clear_face\picture90_6574_.png
D:\newfolder43_1\clear_face\picture90_6576_.png
D:\newfolder43_1\clear_face\picture90_6578_.png
D:\newfolder43_1\clear_face\picture90_6579_.png
D:\newfolder43_1\clear_face\picture90_65

D:\newfolder43_1\clear_face\picture90_8248_.png
D:\newfolder43_1\clear_face\picture90_8257_.png
D:\newfolder43_1\clear_face\picture90_8263_.png
D:\newfolder43_1\clear_face\picture90_8265_.png
D:\newfolder43_1\clear_face\picture90_8284_.png
D:\newfolder43_1\clear_face\picture90_8287_.png
D:\newfolder43_1\clear_face\picture90_8290_.png
D:\newfolder43_1\clear_face\picture90_8291_.png
D:\newfolder43_1\clear_face\picture90_8292_.png
D:\newfolder43_1\clear_face\picture90_8296_.png
D:\newfolder43_1\clear_face\picture90_8297_.png
D:\newfolder43_1\clear_face\picture90_8351_.png
D:\newfolder43_1\clear_face\picture90_8356_.png
D:\newfolder43_1\clear_face\picture90_8362_.png
D:\newfolder43_1\clear_face\picture90_8391_.png
D:\newfolder43_1\clear_face\picture90_8395_.png
D:\newfolder43_1\clear_face\picture90_8402_.png
D:\newfolder43_1\clear_face\picture90_8405_.png
D:\newfolder43_1\clear_face\picture90_8421_.png
D:\newfolder43_1\clear_face\picture90_8449_.png
D:\newfolder43_1\clear_face\picture90_84

In [ ]:
#网络model
class FC_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super(FC_Net,self).__init__()
        self.layer1 = nn.Sequential(
                                    
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )

    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [ ]:
class txt_dataset(data.Dataset):
    def __init__(self,root,transforms0=None):
        self.paths = [root+"\\"+path+"\\"+val for path in os.listdir(root) for val in os.listdir(root+"\\" +path)]
        self.transforms0 = transforms0
        
    def __getitem__(self,index):   
        data_path = self.paths[index]
        label = 1 if 'clear_face' in data_path.split("\\")[-2] else 0
        fr = open(data_path)
        data_str = fr.read()
        array = np.array(data_str.split(",")).astype(float)
        fr.close()
        data = t.from_numpy(array)        
        
        return data,label
    
    def __len__(self):
        return len(self.paths)
                                        

In [ ]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 500
weight_decay = 1e-4

In [ ]:
dataset0 = txt_dataset("C:\\newfolder43_1_txt7")
dataset0[0][1]


In [ ]:
dataloader = DataLoader(dataset0,batch_size=batch_size,shuffle=True)
#dataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#labels

In [ ]:
input_dim=1344
n_hidden_1=128
n_hidden_2=64
n_hidden_3=32
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [ ]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay) 

In [ ]:
print("training start time  <===>  ",time.localtime())
sum_loss = t.Tensor([0])
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):
        data_batch, label = data        
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward() 
        optimizer.step()
        
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='blur_new7',opts={'title': 'loss-batch_new7', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\new_model_save\\FC_500_128_64_NoSoftmax_4x4x28x3std_NEWguiyihua_txt_lr1e-4.pth") 
print("training start time  <===>  ",time.localtime())